# Data Communications End Semester Project

Run use `Cell > Rull All` to initiate the interactive widget states

## Assignment 2

In [1]:
import numpy as np
import matplotlib.pyplot as plot
import ipywidgets as widgets

In [2]:
BITRATE = 10

## NRZ-L Encoding

### Theory

NRZ-L Encoding is a line encoding technique, specifically a serializer line code used to send information bitwise. Conventionally, `1` is represented by one physical level `-1`, while `0` is represented by another level `1`.

In bipolar NRZL encoding, the signal essentially _swings_ from one level to another.

In [3]:
def plot_nrzl(s, noise, freq):
    sampling_freq = freq // BITRATE

    noise_signal = np.random.normal(0, noise, sampling_freq * len(s))

    input_signal = []
    encoded_signal = []
    encoded_signal_with_noise = []
    decoded_signal = []
    decoded_signal_from_noise = []

    input_signal = list(map(int, list(s)))

    for bit in input_signal:
        if bit == 0:
            encoded_signal.append(1)
        else:
            encoded_signal.append(-1)

    encoded_signal_with_noise = [
        i for i in encoded_signal for j in range(sampling_freq)] + noise_signal

    for i in range(len(s)):
        bit_sum = 0
        for j in range(sampling_freq):
            bit_sum += encoded_signal_with_noise[i * sampling_freq + j]
        if bit_sum // sampling_freq < 0:
            decoded_signal_from_noise.append(0)
        else:
            decoded_signal_from_noise.append(1)

    for i in range(len(s)):
        if decoded_signal_from_noise[i] == 1:
            decoded_signal.append(0)
        else:
            decoded_signal.append(1)

    bit_error_count = 0

    for i in range(len(s)):
        if input_signal[i] != decoded_signal[i]:
            bit_error_count += 1

    bit_error_rate = bit_error_count / len(s) * 100

    time = np.arange(0, sampling_freq * len(s))

    (fig, axes) = plot.subplots(ncols=2, sharex=True, sharey=True,
                                figsize=(15, 4), squeeze=False)

    axes[0][0].step(time, [bit for bit in encoded_signal for j in range(
        sampling_freq)], 'r', label='NRZL')
    axes[0][0].plot(time, encoded_signal_with_noise, 'g',
                    label='NRZL w/ Noise')
    axes[0][1].step(time, [bit for bit in decoded_signal for j in range(
        sampling_freq)], 'b', label='Decoded signal')

    axes[0][0].legend()
    axes[0][1].legend()

    print("Bit errors =", bit_error_count)
    print("BER =", bit_error_rate)

## NRZ-I Encoding

### Theory

NRZ-I Encoding is another serializer line encoding technique, used to send information bitwise.

The two-level NRZI signal distinguishes data bits by the presence or absence of a transition, meaning that a `1` is represented by a transition from the previous encoded bit, while `0` is represented by no transition.

NRZ-I encoding is used in USBs, but the opposite convention i.e. "change on 0" is used for encoding.


In [4]:
def plot_nrzi(s, noise, freq):
    sampling_freq = freq // BITRATE

    noise_signal = np.random.normal(0, noise, sampling_freq * len(s))

    input_signal = []
    encoded_signal = []
    encoded_signal_with_noise = []
    decoded_signal = []
    decoded_signal_from_noise = []

    input_signal = list(map(int, list(s)))

    last_bit = 0

    for bit in input_signal:
        if bit == 1:
            last_bit = (1 if last_bit == 0 else 0)
        encoded_signal.append(last_bit)

    encoded_signal_with_noise = [i for i in encoded_signal for j in
                                 range(sampling_freq)] + noise_signal

    for i in range(len(s)):
        bit_sum = 0
        for j in range(sampling_freq):
            bit_sum += encoded_signal_with_noise[i * sampling_freq + j]
        if bit_sum // sampling_freq < 0.5:
            decoded_signal_from_noise.append(0)
        else:
            decoded_signal_from_noise.append(1)

    for i in range(1, len(s)):
        if decoded_signal_from_noise[i] == decoded_signal_from_noise[i - 1]:
            decoded_signal.append(0)
        else:
            decoded_signal.append(1)

    if encoded_signal_with_noise[0] == 0:
        decoded_signal.insert(0, 0)
    else:
        decoded_signal.insert(0, 1)

    bit_error_count = 0

    for i in range(len(s)):
        if input_signal[i] != decoded_signal[i]:
            bit_error_count += 1

    bit_error_rate = bit_error_count / len(s) * 100

    time = np.arange(0, sampling_freq * len(s))

    (fig, axes) = plot.subplots(ncols=2, sharex=True, sharey=True,
                                figsize=(15, 4), squeeze=False)

    axes[0][0].step(time, [bit for bit in encoded_signal for j in
                           range(sampling_freq)], 'r', label='NRZI')
    axes[0][0].plot(time, encoded_signal_with_noise, 'g',
                    label='NRZI w/ Noise')
    axes[0][1].step(time, [bit for bit in decoded_signal for j in
                           range(sampling_freq)], 'b', label='Decoded signal')

    axes[0][0].legend()
    axes[0][1].legend()

    print("Bit errors =", bit_error_count)
    print("BER =", bit_error_rate)

In [5]:
def plot_input(s, freq):
    sampling_freq = freq // BITRATE

    time = np.arange(0, sampling_freq * len(s))

    plot.step(time, [bit for bit in list(map(int, list(s))) for j in
                     range(sampling_freq)], 'b')

    plot.title('Input data')
    plot.figure(figsize=(15, 7))

In [6]:
bitstring = widgets.Text(
    value="110101001010",
    description='Input bitstring'
)

nrzl_n = widgets.FloatSlider(
    value=2,
    min=0, max=4, step=0.25,
    description='Noise Threshold'
)

nrzi_n = widgets.FloatSlider(
    value=0.75,
    min=0, max=1.5, step=0.1,
    description='Noise Threshold'
)

s_f = widgets.IntSlider(
    value=150, step=10,
    min=20, max=200,
    description='Sampling Frequency'
)

widgets.interact(plot_input, s=bitstring, freq=s_f)
widgets.interact(plot_nrzl, s=bitstring, noise=nrzl_n, freq=s_f)
widgets.interact(plot_nrzi, s=bitstring, noise=nrzi_n, freq=s_f)

interactive(children=(Text(value='110101001010', description='Input bitstring'), IntSlider(value=150, descript…

interactive(children=(Text(value='110101001010', description='Input bitstring'), FloatSlider(value=2.0, descri…

interactive(children=(Text(value='110101001010', description='Input bitstring'), FloatSlider(value=0.75, descr…

<function __main__.plot_nrzi(s, noise, freq)>

### Analysis

We can clealy see that both encodings require different thresholds for decoding, because NRZ-L encoding transitions from 0 to 1, requiring a threshold of 0.5, while NRZ-I required a  threshold of 0.

The signal may become asynchronous without an explicit clock signal provided along with the encoding, especially in scenarios when long, unchanges bits are sent as input. Other types of encodings like the Manchester encoding overcome this problem, but require much more bandwidth for the same.

A greater noise threshold brings in more bit errors and subsequently a greater BER, while a higher sampling frequency reduces the BER.